In [ ]:

!pip install -U --upgrade --quiet langchain-google-vertexai langchain-google-genai langchain-community langchain unstructured lark chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

Installing all necessary python modules

In [ ]:
!pip show langchain langchain-core

Name: langchain
Version: 0.1.14
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-core
Version: 0.1.37
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, requests, tenacity
Required-by: langchain, langchain-community, langchain-google-genai, langchain-google-vertexai, langchain-text-splitters


In [ ]:
import os
import google.generativeai as genai
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI


from IPython.display import display
from IPython.display import Markdown

os.environ["GOOGLE_API_KEY"] = userdata.get('KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

Setting up the environment, connecting my API key to be able to access the Gemini through the Colab environment

Want to get a list of models that I can use....

In [ ]:
models = [m for m in genai.list_models()]
models

[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtai

Testing that the API keys are in fact working and that I can access/test Gemini from within this environment

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

result = model.invoke("Waht is an LLM?")

Markdown(result.content)

Can begin actually working on the chaining

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Define the initial prompt template for generating project architecture
architecture_prompt_template = """Given the project description below, generate an initial architecture outline suitable for GCP using Terraform. Keep it concise and to the point.

Project Description:
{project_description}

User Architecture Feedback:
{user_architecture_feedback}

Initial Architecture Outline:"""

# Define the prompt template for generating Terraform code based on confirmed architecture
terraform_prompt_template = """
Given the confirmed architecture outline below, generate the full Terraform code necessary to deploy the project on GCP. The generated code should reflect the unique requirements of the specified architecture, focusing on creating a scalable and secure infrastructure. Use the example below as a guide for the format and structure but develop your own resource configurations and names based on the project's needs. Your code should include a Compute Engine instance, appropriate networking setup, and any other resources deemed necessary for a basic web application.

Note: The example is for illustration only. Please generate custom configurations based on the architecture outline.

Example Terraform Code:
resource "google_compute_instance" "default" {{
  name         = "example-instance"
  machine_type = "e2-medium"
  zone         = "us-central1-a"

  boot_disk {{
    initialize_params {{
      image = "debian-cloud/debian-9"
    }}
  }}

  network_interface {{
    network = "default"
    access_config {{}}
  }}
}}

Confirmed Architecture Outline:
{{confirmed_architecture}}

Existing Terraform Code:
{{existing_terraform_code}}

User Feedback:
{{user_terraform_feedback}}

Your Terraform Code:
"""


# Custom prompt templates
architecture_prompt = PromptTemplate.from_template(
    architecture_prompt_template)
terraform_code_prompt = PromptTemplate.from_template(
    terraform_prompt_template)

# Define the interaction chain for architecture generation and confirmation
architecture_chain = ({
    "project_description": RunnablePassthrough(),
    "user_architecture_feedback": RunnablePassthrough()
}
    | architecture_prompt
    | model
    | StrOutputParser())

# Define the interaction chain for Terraform code generation
terraform_code_chain = ({
    "confirmed_architecture": RunnablePassthrough(),
    "existing_terraform_code": RunnablePassthrough(),
    "user_terraform_feedback": RunnablePassthrough()
}
    | terraform_code_prompt
    | model
    | StrOutputParser())

# Define the prompt template for suggesting improvements
improvement_prompt_template = """
Given the confirmed architecture outline below, suggest potential improvements that could enhance the project's efficiency, scalability, or reliability.

Confirmed Architecture Outline:
{confirmed_architecture}

Suggested Improvements:"""

improvement_prompt = PromptTemplate.from_template(
    improvement_prompt_template)

# Define the interaction chain for improvement suggestions
improvement_chain = ({
    "confirmed_architecture": RunnablePassthrough()
}
    | improvement_prompt
    | model
    | StrOutputParser())

project_description = input("Enter the project description: ")

user_architecture_feedback = "Initial Feedback (None)"
user_terraform_feedback = "Initial Feedback (None)"
existing_terraform_code = ""

architecture = None
while not architecture:
  try:
    architecture = architecture_chain.invoke({
        "project_description": project_description,
        "user_architecture_feedback": "user_architecture_feedback"
    })
  except Exception as e:
    print("Error:", e)
    break

  print("Generated Architecture: ", architecture)
  user_confirmation = input("User confirms this architecture? (yes/no): ")
  if user_confirmation.lower() == 'yes':
    break
  else:
    user_architecture_feedback = input("Please describe the issues you have with the architecture: ")

improvements = improvement_chain.invoke(
  {"confirmed_architecture": architecture})

# Display the suggested improvements to the user
print("Suggested Improvements: ", improvements)
user_confirmation = input(
  "Would you like to incorporate suggested improvements? (yes/no): ")

if user_confirmation.lower() == 'yes':
    # Update the architecture with the improvements
    architecture += "\n\nIncorporated Improvements:\n" + improvements

# print()
# print(architecture)
terraform_code_correct = False
terraform_code_generated = None
while terraform_code_correct == False:
  print("Architecture: ", architecture)
  try:
    terraform_code_generated = terraform_code_chain.invoke({
        "confirmed_architecture": architecture,
        "existing_terraform_code": existing_terraform_code,
        "user_terraform_feedback": user_terraform_feedback
    })
  except Exception as e:
    print("Error:", e)
    break

  print("Generated Terraform Code: ", terraform_code_generated)
  user_confirmed_terraform_code = input("User confirms this Terraform code? (yes/no): ")
  if user_confirmed_terraform_code.lower() == 'yes':
    terraform_code_correct = True
  else:
      existing_terraform_code = terraform_code_generated
      user_terraform_feedback = input("What is wrong with the code? ")




Enter the project description: Generate me terraform code that will allow me to host a website on GCP
Generated Architecture:  **Infrastructure**

* **Virtual Machine:** Host the website's application and content.
* **Cloud Storage:** Store website files and assets.
* **Load Balancer:** Distribute traffic to the virtual machine.
* **DNS Management:** Manage domain name records.

**Networking**

* **Virtual Private Cloud (VPC):** Isolate the website's network resources.
* **Firewall Rules:** Control inbound and outbound traffic.

**Management**

* **Service Account:** Authenticate Terraform operations.
* **IAM Roles:** Assign permissions for resource management.
* **Monitoring:** Configure metrics and alerts for website performance.

**Terraform Modules**

* **Virtual Machine:** Create and configure the virtual machine.
* **Cloud Storage:** Create and configure the Cloud Storage bucket.
* **Load Balancer:** Create and configure the load balancer.
* **DNS Management:** Manage DNS records

KeyboardInterrupt: Interrupted by user